In [1]:
library(lme4)
library(lmerTest)
library(stringr)

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step




In [6]:
data_profile <- read.csv('/Users/m221138/RA_plasma_metabolites/data/67_met_every_confounds_log10.csv', row.names = 1, header=TRUE, sep=",")
data_profile <- as.data.frame(t(data_profile))
output_drug_file <- '/Users/m221138/RA_plasma_metabolites/data/67_met_every_drug_log10.results.tsv'
output_confounding_file <- '/Users/m221138/RA_plasma_metabolites/data/67_met_every_confounds_log10.results.tsv'
output_confounding_v2_file <- '/Users/m221138/RA_plasma_metabolites/data/8_met_confounds_log10.results.tsv'


In [7]:
head(data_profile)

,patient_dummyID,SEX,AGE,mtx,csdmard,tnfi,ntnfi,pred,smoke,crp,⋯,100002911,1489,100020361,100001104,100000269,100020866,100001510,100001767,171,344
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ST.00010415_t1,0,1,75,1,0,0,0,0,1,0.4623980,⋯,1.6174,5.8065,1.5960,2.3718,0.7689,4.2787,2.8148,1.5577,0.0243,1.3874
ST.00010415_t2,0,1,76,1,0,0,0,0,1,0.4623980,⋯,0.9193,0.5310,1.1545,1.4633,0.9511,4.7451,2.6375,0.8981,0.2159,1.3073
ST.00010421_t1,1,1,70,1,0,1,0,1,1,0.6334685,⋯,1.3136,7.5976,1.3427,1.1518,1.3642,1.7638,0.7221,3.1478,0.1789,1.3242
ST.00010421_t2,1,1,71,1,0,1,0,1,1,0.4623980,⋯,3.3806,0.4750,0.9460,0.8480,0.7717,1.4401,1.4132,0.5696,0.4390,1.1715
ST.00011458_t1,2,0,69,0,1,0,0,0,0,0.4623980,⋯,0.3923,2.7211,1.1114,0.8064,2.3825,0.9498,0.9553,1.2109,0.1584,1.0916
ST.00011458_t2,2,0,69,0,1,0,0,0,0,0.4623980,⋯,2.5219,0.4712,1.0083,0.8661,1.2101,0.9348,1.5982,0.3697,1.0086,1.5167


In [8]:
NUM_CHEM_ID <- ncol(data_profile)

In [9]:
if (file.exists(output_confounding_file)){
    file.remove(output_confounding_file)
}

if (file.exists(output_confounding_v2_file)){
    file.remove(output_confounding_v2_file)
}

cat("chemID\tEveryConfound(re)\tAGE(re)\tSEX(re)\tsmoke(re)\tMTX(re)\tPRED(re)\tntnfi(re)\ttnfi(re)\tcsdmard(re)\tcrp(re)\n", file=output_confounding_file,append=TRUE)
cat("chemID\tpvalue\n", file=output_confounding_v2_file,append=TRUE)
for (i in 1:NUM_CHEM_ID){
    if (i > 10){
        
    chemID <- colnames(data_profile)[i]
    chem_profile <- data_profile[,i]
        
    #EVERYTHING
    all_m <- lmer(chem_profile ~ crp + SEX + mtx + smoke + pred + ntnfi + tnfi + csdmard + AGE + (1|patient_dummyID), data = data_profile)    
    print (anova(all_m))
    all_p <- anova(all_m)$Pr[1]
    
    age_m <- lmer(chem_profile ~ AGE + (1|patient_dummyID), data = data_profile)
    age_p <- anova(age_m)$Pr[1]
        
    sex_m <- lmer(chem_profile ~ SEX + (1|patient_dummyID), data = data_profile)
    sex_p <- anova(sex_m)$Pr[1]
        
    smoke_m <- lmer(chem_profile ~ smoke + (1|patient_dummyID), data = data_profile)
    smoke_p <- anova(smoke_m)$Pr[1]
        
    mtx_m <- lmer(chem_profile ~ mtx + (1|patient_dummyID), data = data_profile)
    mtx_p <- anova(mtx_m)$Pr[1]
        
    pred_m <- lmer(chem_profile ~ pred + (1|patient_dummyID), data = data_profile)
    pred_p <- anova(pred_m)$Pr[1]
        
    ntnfi_m <- lmer(chem_profile ~ ntnfi + (1|patient_dummyID), data = data_profile)
    ntnfi_p <- anova(ntnfi_m)$Pr[1]   
        
    tnfi_m <- lmer(chem_profile ~ tnfi + (1|patient_dummyID), data = data_profile)
    tnfi_p <- anova(tnfi_m)$Pr[1]
           
    csdmard_m <- lmer(chem_profile ~ csdmard + (1|patient_dummyID), data = data_profile)
    csdmard_p <- anova(csdmard_m)$Pr[1]
        
    crp_m <- lmer(chem_profile ~ crp + (1|patient_dummyID), data = data_profile)
    crp_p <- anova(crp_m)$Pr[1]
    
    output_string <- paste(chemID, all_p, age_p, sex_p, smoke_p, mtx_p, pred_p, ntnfi_p, tnfi_p, csdmard_p, crp_p, "\n", sep="\t")
    cat(output_string, file=output_confounding_file,append=TRUE)
     
    if (chemID == 100000007){
    task1_m <- lmer(chem_profile ~ crp + pred + ntnfi + (1|patient_dummyID), data = data_profile)    
    task1_p <- anova(task1_m)$Pr[1]
    output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
    cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
    }
    if (chemID == 100020361){
    task1_m <- lmer(chem_profile ~ crp + AGE + ntnfi + tnfi + (1|patient_dummyID), data = data_profile)    
    task1_p <- anova(task1_m)$Pr[1]
    output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
    cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
    }    

    if (chemID == 407){
    task1_m <- lmer(chem_profile ~ crp + ntnfi + (1|patient_dummyID), data = data_profile)    
    task1_p <- anova(task1_m)$Pr[1]
    output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
    cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
    }
        
    if (chemID == 100001266){
    task1_m <- lmer(chem_profile ~ crp + pred + (1|patient_dummyID), data = data_profile)    
    task1_p <- anova(task1_m)$Pr[1]
    output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
    cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
    } 
        
    if (chemID == 344){
    task1_m <- lmer(chem_profile ~ crp + SEX + smoke + ntnfi + (1|patient_dummyID), data = data_profile)    
    task1_p <- anova(task1_m)$Pr[1]
    output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
    cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
    }        
        
    if (chemID == 803){
    task1_m <- lmer(chem_profile ~ crp + smoke + mtx + ntnfi + (1|patient_dummyID), data = data_profile)    
    task1_p <- anova(task1_m)$Pr[1]
    output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
    cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
    }    
        
    if (chemID == 1162){
    task1_m <- lmer(chem_profile ~ crp + ntnfi + (1|patient_dummyID), data = data_profile)    
    task1_p <- anova(task1_m)$Pr[1]
    output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
    cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  
    }        

    if (chemID == 100001586){
    task1_m <- lmer(chem_profile ~ crp + AGE + tnfi + (1|patient_dummyID), data = data_profile)    
    task1_p <- anova(task1_m)$Pr[1]
    output_v2_string <- paste(chemID, task1_p, "\n", sep="\t")
    cat(output_v2_string, file=output_confounding_v2_file, append=TRUE)  

    }   
     
}}

print ("Done")

[1] TRUE

[1] TRUE

boundary (singular) fit: see ?isSingular



Type III Analysis of Variance Table with Satterthwaite's method
        Sum Sq Mean Sq NumDF DenDF F value  Pr(>F)  
crp      3.845   3.845     1   117  0.5067 0.47798  
SEX     36.629  36.629     1   117  4.8278 0.02997 *
mtx      2.309   2.309     1   117  0.3044 0.58221  
smoke   18.308  18.308     1   117  2.4131 0.12303  
pred     6.212   6.212     1   117  0.8187 0.36742  
ntnfi    0.061   0.061     1   117  0.0080 0.92870  
tnfi     3.034   3.034     1   117  0.3999 0.52838  
csdmard 13.982  13.982     1   117  1.8429 0.17723  
AGE      4.862   4.862     1   117  0.6408 0.42505  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular



...

boundary (singular) fit: see ?isSingular



Type III Analysis of Variance Table with Satterthwaite's method
         Sum Sq Mean Sq NumDF DenDF F value  Pr(>F)  
crp     1.88183 1.88183     1   117  6.0998 0.01496 *
SEX     0.24781 0.24781     1   117  0.8033 0.37196  
mtx     0.17596 0.17596     1   117  0.5704 0.45163  
smoke   0.24209 0.24209     1   117  0.7847 0.37752  
pred    0.63880 0.63880     1   117  2.0706 0.15283  
ntnfi   0.05220 0.05220     1   117  0.1692 0.68158  
tnfi    0.19185 0.19185     1   117  0.6219 0.43195  
csdmard 0.48291 0.48291     1   117  1.5653 0.21338  
AGE     0.01519 0.01519     1   117  0.0492 0.82479  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular

boundary (singular) fit: see ?isSingular



Type III Analysis of Variance Table with Satterthwaite's method
        Sum Sq Mean Sq NumDF   DenDF F value   Pr(>F)   
crp     1.8272  1.8272     1 116.200  2.5606 0.112272   
SEX     0.4711  0.4711     1  61.664  0.6602 0.419630   
mtx     0.1648  0.1648     1 105.196  0.2310 0.631789   
smoke   3.5431  3.5431     1  76.673  4.9652 0.028793 * 
pred    0.0758  0.0758     1 115.472  0.1062 0.745101   
ntnfi   0.0234  0.0234     1 110.825  0.0328 0.856552   
tnfi    5.2472  5.2472     1 116.783  7.3533 0.007705 **
csdmard 0.1875  0.1875     1 103.450  0.2627 0.609353   
AGE     1.3243  1.3243     1  61.343  1.8558 0.178091   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
Type III Analysis of Variance Table with Satterthwaite's method
         Sum Sq Mean Sq NumDF   DenDF F value   Pr(>F)   
crp     0.11374 0.11374     1 113.374  0.5800 0.447877   
SEX     0.39549 0.39549     1  56.279  2.0169 0.161077   
mtx     0.08803 0.08803     1  88.183  0.4489 0.504589   
smo

[1] "Done"
